In [ ]:
"""
Sentiment Analysis_Logistics Regression 
# Adapted from Kaggle (https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews).
# Implemented  by Lavanya Thollamadugu
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from bs4 import BeautifulSoup
import nltk
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import warnings
warnings.filterwarnings("ignore")

**Collect tweets for a specific movie** 

In [ ]:
import tweepy
from tweepy import *
import pandas as pd
import csv
import string
 
consumer_key = "pDI4tjiuK0cxDZzUbiYul0bp1"
consumer_secret = "W6gbnpZdClvVK2yO2Yvys3BxCPgTVog5xqfHSQ1oIOE6eR4vNw"
access_key= "1567948456341426178-VJ3JXCzWoxqaKDZGinWnVoPYtvBsOa"
access_secret = "qNu4ARlIHMQo67dqwAaHuJzwQdXwuG0MCvp4vq05LGpKH"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
 
api = tweepy.API(auth,wait_on_rate_limit=True)
 
csvFile = open('/content/AVATAR.csv', 'w')
csvWriter = csv.writer(csvFile)
search_words = "#AVATAR"      # enter your words
new_search = search_words + " -filter:retweets"
 
for tweet in tweepy.Cursor(api.search,q=new_search,count=10000,
                           lang="en",
                           since_id=0).items(10000):
    csvWriter.writerow([tweet.text.encode('utf-8')])

In [ ]:
df_tweets = pd.read_csv('/content/AVATAR.csv',skiprows=1,names=['review'])

In [ ]:
df_tweets.size

4227

In [ ]:
df_tweets.drop(df_tweets.tail(27).index,
        inplace = True)

In [ ]:
df_tweets.shape

(4170, 1)

**Data Preprocessing** 

In [ ]:
def process(review):
    # remove  HTML tags
    review = BeautifulSoup(review).get_text()
    # remove punctuation and numbers
    review = re.sub("[^a-zA-Z]",' ',review)
    # converting into lowercase and splitting to eliminate stopwords
    review = review.lower()
    review = review.split()
    # remove stopwords
    swords = set(stopwords.words("english"))  
    review = [w for w in review if w not in swords]      
    return(" ".join(review))

 #download the stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#clean the twitter data
twitter_data = []
for i in range(len(df_tweets["review"])):        
    twitter_data.append(process(df_tweets["review"][i]))

len(twitter_data)

4170

In [ ]:
#stemming function
ps = PorterStemmer()

def perform_stemming(text):
    new_text = [ps.stem(word) for word in text.split()]
    return ' '.join(new_text)

In [ ]:
for i in range(len(twitter_data[i])):
     twitter_data[i] = perform_stemming(twitter_data[i])

In [ ]:
# we convert twitter  data to feature vector matrix
cv=CountVectorizer(max_features=1000)
cv_twitter_data=cv.fit_transform(twitter_data)
cv_twitter_data.shape

In [ ]:
#import the IMDB movire reviews data
from google.colab import drive
drive.mount('/content/drive')
df_data= pd.read_csv('/content/drive/My Drive/IMDB Dataset.csv')
#df_data= pd.read_csv('/content/IMDB Dataset.csv')
df_data.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(50000, 2)

In [ ]:
df_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [ ]:
#clean the IMDB data
train_data = []
for i in range(len(df_data["review"])):        
    train_data.append(process(df_data["review"][i]))

In [ ]:
for i in range(len(train_data[i])):
     train_data[i] = perform_stemming(train_data[i])

**Splitting to train and test data** 

In [ ]:
label = np.array(df_data["sentiment"])

train_x, test_x, y_train, y_test = train_test_split(train_data, label, stratify = label ,test_size = 0.2, shuffle = True , random_state = 25)

In [ ]:
train_positive_reviews=y_train.tolist().count('positive')
train_negative_reviews=y_train.tolist().count('negative')
test_positive_reviews=y_test.tolist().count('positive')
test_negative_reviews=y_test.tolist().count('negative')
print("Positive reviews in the train dataset : ",train_positive_reviews)
print("Negative reviews in the train dataset : ",train_negative_reviews)
print("Positive reviews in the test dataset : ",test_positive_reviews)
print("Negative reviews in the test dataset : ",test_negative_reviews)

Positive reviews in the train dataset :  20000
Negative reviews in the train dataset :  20000
Positive reviews in the test dataset :  5000
Negative reviews in the test dataset :  5000


In [ ]:
vectorizer = CountVectorizer( max_features = 1000 )

# convert IMDB  data to feature vector matrix
train_x = vectorizer.fit_transform(train_x)

train_x = train_x.toarray()
train_y = y_train

print("Total training data :", train_x.shape[0])
print("Total training data :", train_y.shape[0])

Total training data : 40000
Total training data : 40000


In [ ]:
test_result = vectorizer.transform(test_x)
test_result = test_result.toarray()

**Logistic regression model** 

In [ ]:
model = LogisticRegression(penalty='l2')
print('-----------------------------------------------------------------------')
print('Logistic Regression')
model.fit(train_x, train_y)
test_pred = model.predict(test_result)
print('Accuracy Score : ',accuracy_score(y_test, test_pred))
print('F1 Score : ',f1_score(y_test, test_pred, average='weighted'))
print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------
Logistic Regression
Accuracy Score :  0.8608
F1 Score :  0.8607518257016196
-----------------------------------------------------------------------


**Sentiment analysis on movie reviews from tweets**

In [ ]:
twitter_pred = model.predict(cv_twitter_data)

In [ ]:
counts = 0
for i in range(len(twitter_pred)):
  if twitter_pred[i] == 'positive':
    counts += 1

positive_rate = "{:.0%}".format(counts/len(twitter_pred))

In [ ]:
print('positive rate:' + positive_rate)

positive rate:32%
